In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import argparse

from habitat.datasets import make_dataset
from ss_baselines.av_nav.config import get_config
from ss_baselines.common.environments import AudioNavRLEnv

config = get_config(
    config_paths="ss_baselines/av_nav/config/audionav/mp3d/env_test_0.yaml", # RGB + AudiogoalSensor
    # opts=["CONTINUOUS", "True"],
    run_type="eval")
config.defrost()
config.TASK_CONFIG.TASK.MEASUREMENTS.append("TOP_DOWN_MAP") # Note: can we add audio sensory info fields here too ?
config.TASK_CONFIG.DATASET.SPLIT = config.EVAL.SPLIT
config.freeze()
print(config)

/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


BASE_TASK_CONFIG_PATH: configs/audionav/av_nav/mp3d/base_env_test.yaml
CHECKPOINT_FOLDER: data/models/output/data
CHECKPOINT_INTERVAL: 50
CMD_TRAILING_OPTS: []
CONTINUOUS: True
DEBUG: True
DISPLAY_RESOLUTION: 128
ENV_NAME: AudioNavRLEnv
EVAL:
  SPLIT: val_telephone
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/models/output/data
EXTRA_RGB: False
FOLLOW_SHORTEST_PATH: False
LOG_FILE: data/models/output/train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 1
NUM_UPDATES: 300000
RL:
  DISTANCE_REWARD_SCALE: 1.0
  PPO:
    clip_param: 0.1
    entropy_coef: 0.2
    eps: 1e-05
    gamma: 0.99
    hidden_size: 512
    lr: 0.00025
    max_grad_norm: 0.5
    num_mini_batch: 1
    num_steps: 150
    ppo_epoch: 4
    reward_window_size: 50
    tau: 0.95
    use_gae: True
    use_linear_clip_decay: True
    use_linear_lr_decay: True
    value_loss_coef: 0.5
  SLACK_REWARD: -0.01
  SUCCESS_REWARD: 10.0
  TIME_DIFF: False
  WITH_DISTANCE_REWARD: True
  WITH_TIME_PENALTY: True
SEED: 0
SENSORS: ['RGB_SENSOR']


In [2]:
dataset = make_dataset(id_dataset=config.TASK_CONFIG.DATASET.TYPE, config=config.TASK_CONFIG.DATASET)
env = AudioNavRLEnv(config=config, dataset=dataset)

2022-07-25 11:25:18,513 Initializing dataset AudioNav
2022-07-25 11:25:18,539 initializing sim ContinuousSoundSpacesSim
2022-07-25 11:25:42,535 Initializing task AudioNav


[11:25:18:547276]:[Metadata] AttributesManagerBase.h(380)::createFromJsonOrDefaultInternal : <Dataset>: Proposing JSON name : default.scene_dataset_config.json from original name : default| This file does not exist.
[11:25:18:548137]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DSolid:capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
[11:25:18:548192]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DWireframe:capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
[11:25:18:548226]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneSolid:coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTangents_false_capEnd_true) created and registered.
[11:25:18:548256]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneWireframe:coneWireframe_s

In [3]:
observation = env.reset()

[11:25:43:366304]:[Sensor] AudioSensor.cpp(81)::setAudioSourceTransform : [Audio]  Setting the audio source position :  Vector(-6.43414, -1.46512, -21.2952) ]
[11:25:43:769501]:[Sensor] AudioSensor.cpp(90)::setAudioListenerTransform : [Audio]  Setting the agent transform : position [ Vector(-3.44611, -1.62041, -18.2299) ], rotQuat[ Vector(1, 0, 2.45858e-05, 0) ]
[11:25:43:769562]:[Sensor] AudioSensor.cpp(278)::createAudioSimulator : [Audio]  Create audio simulator iteration:  0
[11:25:43:769915]:[Sensor] AudioSensor.cpp(114)::runSimulation : [Audio]  Running the audio simulator
[11:25:43:769928]:[Sensor] AudioSensor.cpp(119)::runSimulation : [Audio]  New initialization, will upload geometry and add the source at position :  Vector(-6.43414, -1.46512, -21.2952)
[11:25:43:769944]:[Sensor] AudioSensor.cpp(126)::runSimulation : [Audio]  Loading semantic scene
[11:25:43:769951]:[Sensor] AudioSensor.cpp(293)::loadSemanticMesh : [Audio]  Loading semantic mesh
[11:25:46:621207]:[Sensor] AudioS

In [4]:
# type(observation) # habitat.core.simulator.Observations
list(observation.keys()) # ['rgb', 'spectrogram'] or 
# observation["rgb"].shape # (128, 128, 3)
# observation["spectrogram"].shape # (65, 26, 2), available when TASK.SENSORS = ["SPECTROGRAM_SENSOR"]
# observation["audiogoal"].shape # (2, 16000), Available wwhen TASK.SENSORS = ['AUDIOGOAL_SENSOR']
# env.action_space # ActionSpace(MOVE_FORWARD:EmptySpace(), STOP:EmptySpace(), TURN_LEFT:EmptySpace(), TURN_RIGHT:EmptySpace())
# action = env.action_space.sample()

['rgb', 'audiogoal', 'spectrogram']

In [ ]:
# fig.imshow(observation["rgb"])

In [ ]:
action_fwd = {"action": "MOVE_FORWARD", "action_args": None}
observation, reward, done, info = env.step(**action_fwd)

In [ ]:
# list(observation.keys())
# reward
# done
# list(info.keys()) # ['distance_to_goal', 'normalized_distance_to_goal', 'success', 'spl', 'softspl', 'na', 'sna', 'top_down_map']

In [ ]:
## Collect one full episode
observation, done, ep_length = env.reset(), False, 0
ep_observations = [observation]

while not done:
    # The "STOP" action will make the episode finish early, but 
    # this work around is not very efficient.
    action = {"action": 'STOP', "action_args": None}
    while action["action"] == 'STOP':
        action = env.action_space.sample()

    observation, reward, done, info = env.step(**action)
    ep_length += 1

    ep_observations.append(observation)

    print("")
    print("###########################################")
    print(f"# DEBUG: Episode length: {ep_length}")
    print(f"DEBUG: Done value {done}")
    print("###########################################")
    print("")

In [ ]:
# 'info' variable after the end of the peisode: do we get any additional infromation of the full episode ? For example all the RGB frames / audio waves as a nice list ?
list(info.keys())

In [ ]:
len(ep_observations) # 501 for randomly sampled episode
# list(ep_observations[0].keys()) # ['rgb', 'audiogoal']
ep_rgb_observations = [obs["rgb"] for obs in ep_observations]
ep_audiogoal_observations = [obs["audiogoal"] for obs in ep_observations]
# ep_rgb_observations.shape, ep_audiogoal_observations.shape # ((501, 128, 128, 3), (501, 2, 16000))

In [ ]:
import os
from typing import Dict, List, Optional
import moviepy.editor as mpy
from moviepy.audio.AudioClip import CompositeAudioClip, AudioArrayClip

def images_to_video_with_audio(
    images: List[np.ndarray],
    output_dir: str,
    video_name: str,
    audios: List[str],
    sr: int,
    fps: int = 1,
    quality: Optional[float] = 5,
    **kwargs
):
    r"""Calls imageio to run FFMPEG on a list of images. For more info on
    parameters, see https://imageio.readthedocs.io/en/stable/format_ffmpeg.html
    Args:
        images: The list of images. Images should be HxWx3 in RGB order.
        output_dir: The folder to put the video in.
        video_name: The name for the video.
        audios: raw audio files
        fps: Frames per second for the video. Not all values work with FFMPEG,
            use at your own risk.
        quality: Default is 5. Uses variable bit rate. Highest quality is 10,
            lowest is 0.  Set to None to prevent variable bitrate flags to
            FFMPEG so you can manually specify them using output_params
            instead. Specifying a fixed bitrate using ‘bitrate’ disables
            this parameter.
    """
    assert 0 <= quality <= 10
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    video_name = video_name.replace(" ", "_").replace("\n", "_") + ".mp4"

    audio_clips = []
    multiplier = 0.5
    for i, audio in enumerate(audios):
        audio_clip = AudioArrayClip(audio.T[:int(sr * 1 / fps)] * multiplier, fps=sr)
        audio_clip = audio_clip.set_start(1 / fps * i)
        audio_clips.append(audio_clip)
    composite_audio_clip = CompositeAudioClip(audio_clips)
    video_clip = mpy.ImageSequenceClip(images, fps=fps)
    video_with_new_audio = video_clip.set_audio(composite_audio_clip)
    video_with_new_audio.write_videofile(os.path.join(output_dir, video_name))

In [ ]:
config.TASK_CONFIG.SIMULATOR.AUDIO.RIR_SAMPLING_RATE # 16000 for mp3d datset
config.TASK_CONFIG.SIMULATOR.VIEW_CHANGE_FPS # 10 for mp3d by default ?

In [ ]:
images_to_video_with_audio(
    images=ep_rgb_observations,
    output_dir="/tmp/ss-videos",
    video_name="ss_video_dgb",
    audios=ep_audiogoal_observations,
    sr=config.TASK_CONFIG.SIMULATOR.AUDIO.RIR_SAMPLING_RATE, # 16000 for mp3d dataset
    fps=config.TASK_CONFIG.SIMULATOR.VIEW_CHANGE_FPS,
)

In [ ]:
# Testing videoe with audio passing to tensorboard
# from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
import torch as th
writer = SummaryWriter("/tmp/ss-videos/tbtest")
vid_tensor = np.array([[obs["rgb"] for obs in ep_observations]]) # [1, N, H, W, C]
# vid_tensor.shape
vid_tensor = th.Tensor(vid_tensor).permute(0, 1, 4, 3, 2)
vid_tensor.shape
for gstep in range(10):
    # writer.add_video(tag="video", vid_tensor=vid_tensor, global_step=gstep, fps=10)
    writer.add_video("audio_video", "/tmp/ss-videos/ss_video_dgb.mp4", 0)

In [ ]:
# Experimenting with WANDB and audio video logging
import wandb
wandb.init(project="ss-hab-test", entity="dosssman", settings=wandb.Settings(start_method='thread'))
wandb.log({"audio_video": wandb.Video(
                data_or_path="/tmp/ss-videos/ss_video_dgb.mp4")})